8/29- in progress of recreating this project here from my "rough" version that's complete, but quite messy - separating out my work into "chunks" to correspond to DSM

<b>Move This Explanation to README file</b>

This project came from me reading about "all-time high" short ratios for SPY, the S&P 500 Index ETF. The article positioned this as an ominous sign, but it made me wonder:

"What tends to happen to future returns when a stock (or ETF) has a high (or low) short ratio?"

On the one hand, a high short ratio suggests a high level of negative sentiment towards that holding. But, on the other, shorts have to close out their positions at some point, so a high short ratio could also portend future demand, which could be a positive.

I thought this would be a great question to explore personally - and a great way to develop my data scraping/pandas manipulation skills.

My goal is to look at the stocks in the S&P 500 and observe any potential relationship(s) between short interest and future returns.

NOTES:
- I could not find a publicly available "short ratio" data set, so I used a proxy based on daily short volume of trading. Namely, I used the trailing 30 day average of short volume as a percentage of total trading volume. To improve the accuracy of this analysis it could be beneficial to access a data set specifically looking at short shares outstanding, not just daily volume of trading.

In [21]:
# Scrape a list of the S&P 500 companies and their ticker symbols from Wikipedia

# Imports
import urllib.request
from bs4 import BeautifulSoup

# Specify url to scrape
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Open the url request and save 
page = urllib.request.urlopen(url)

# Parse the HTML into BeautifulSoup
soup = BeautifulSoup(page, 'lxml')

# Create a variable to hold the HTML for the table in question (that holds the S&P 500 stocks and tickers)
# From exploring the HTML of the page I found that the table in question was a class 'wikitable sortable'
ticker_table = soup.find('table', class_='wikitable sortable')

# Initialize lists to "catch" the tickers and company names
tickers = []
companies = []

# Loop through my table rows and extract the first two cells of each row (which correspond to 'ticker' and 'company')
for row in ticker_table.find_all('tr'):
    cells = row.find_all('td')
    count = 1
    for cell in cells:
        if count == 1:
            tickers.append(cell.find('a', href=True).string)
            count += 1
        elif count == 2:
            companies.append(cell.string)
            count += 2
        else:
            count +=1

In [41]:
# Check my lists
display(tickers)
display(len(tickers))
display(companies)
display(len(companies))

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'BIIB',
 'BLK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BF.B',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA',
 'CAG',
 'CXO'

505

['3M Company',
 'Abbott Laboratories',
 'AbbVie Inc.',
 'ABIOMED Inc',
 'Accenture plc',
 'Activision Blizzard',
 'Adobe Inc.',
 'Advanced Micro Devices Inc',
 'Advance Auto Parts',
 'AES Corp',
 'AFLAC Inc',
 'Agilent Technologies Inc',
 'Air Products & Chemicals Inc',
 'Akamai Technologies Inc',
 'Alaska Air Group Inc',
 'Albemarle Corp',
 'Alexandria Real Estate Equities',
 'Alexion Pharmaceuticals',
 'Align Technology',
 'Allegion',
 'Alliant Energy Corp',
 'Allstate Corp',
 None,
 None,
 'Altria Group Inc',
 'Amazon.com Inc.',
 'Amcor plc',
 'Ameren Corp',
 'American Airlines Group',
 'American Electric Power',
 'American Express Co',
 'American International Group',
 'American Tower Corp.',
 'American Water Works Company Inc',
 'Ameriprise Financial',
 'AmerisourceBergen Corp',
 'AMETEK Inc.',
 'Amgen Inc.',
 'Amphenol Corp',
 'Analog Devices, Inc.',
 'ANSYS',
 'Anthem',
 'Aon plc',
 'A.O. Smith Corp',
 'Apache Corporation',
 'Apartment Investment & Management',
 'Apple Inc.',
 '

505

The lists look right, but we have 505 comapanies/tickers, NOT 500. Looks like it could be an error, but actually the S&P 500 name is slightly misleading. 

The "500" refers to the number of companies in the index. There are actually 505 different securities listed on the index. Some companies have multiple securities listed - for example: Berkshire Hathaway has both 'A' and 'B' shares listed.

This means our list lengths look good.

In [23]:
# Import pandas and iniatilize dataframe for final data
import pandas as pd
final_df = pd.DataFrame(tickers, columns=['ticker'])
final_df['company'] = companies

In [24]:
# Import and authenticate Quandl and datetime
import datetime
import quandl
quandl.ApiConfig.api_key = 'uB8yxfS6qW2Qc3xNcd9z'

In [35]:
# Import the csv file with all of securities listed in the FINRA short sale dataset
filename = 'finra_short_data.csv'
finra_df = pd.read_csv(filename)
finra_df['to_dat'] = '8-28-2020'

# Check
display(finra_df.head())
display(finra_df.info())

,code,name,description,refreshed_at,from_date,to_date,to_dat
0,FNRA_A,ADF Short Interest: A,This dataset has no description.,11/1/2017 0:22,2/18/2014,11/30/2016,8-28-2020
1,FNRA_AA,ADF Short Interest: AA,This dataset has no description.,11/1/2017 0:22,2/18/2014,11/30/2016,8-28-2020
2,FNRA_AA_,ADF Short Interest: AAp,This dataset has no description.,3/22/2014 1:58,2/24/2014,3/20/2014,8-28-2020
3,FNRA_AAAP,ADF Short Interest: AAAP,This dataset has no description.,11/1/2017 0:22,2/1/2016,11/30/2016,8-28-2020
4,FNRA_AAC,ADF Short Interest: AAC,This dataset has no description.,11/1/2017 0:22,10/2/2014,11/30/2016,8-28-2020


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69458 entries, 0 to 69457
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   code          69458 non-null  object
 1   name          69458 non-null  object
 2   description   69458 non-null  object
 3   refreshed_at  69458 non-null  object
 4   from_date     69458 non-null  object
 5   to_date       69458 non-null  object
 6   to_dat        69458 non-null  object
dtypes: object(7)
memory usage: 3.7+ MB


None

This FINRA dataset (that I'll use for my short ratio data), contains time series data for nearly 70k securities. I'm only going to use the S&P 500 companies scraped above.

The name column includes the company's ticker symbol - which I'll need - but also a lot of extra text that's irrelevant for me.

In [36]:
# Split the name column to break out the tickers
ticker_split = finra_df['name'].str.split(':', expand=True)


# Make a new column in my finra_df with the ticker, then drop uneccesary columns
finra_df['ticker'] = ticker_split[1].str.strip()
finra_df.drop(labels=['name', 'description', 'refreshed_at'], axis=1, inplace=True)

# Check so far
display(finra_df.head())

,code,from_date,to_date,to_dat,ticker
0,FNRA_A,2/18/2014,11/30/2016,8-28-2020,A
1,FNRA_AA,2/18/2014,11/30/2016,8-28-2020,AA
2,FNRA_AA_,2/24/2014,3/20/2014,8-28-2020,AAp
3,FNRA_AAAP,2/1/2016,11/30/2016,8-28-2020,AAAP
4,FNRA_AAC,10/2/2014,11/30/2016,8-28-2020,AAC


In [46]:
# Use finra_df to make calls to quandl.get and gather the short interest data for 
# S&P 500 companies

# Create dataframe for short interest data
short_df = pd.DataFrame()

# Change dates in my finra_df to date objects
finra_df['from_date'] = pd.to_datetime(finra_df['from_date'])
finra_df['to_date'] = pd.to_datetime(finra_df['to_date'])

# format list of S&P 500 tickers to replace '.' with '_'
tickers = [x.replace('.','_') for x in tickers]

In [52]:
# Loop through tickers in my list of S&P 500 tickers and pull that data from Quandl
### This one takes a while to run
for ticker in tickers:
    
    # Format the code I want based on the ticker and for my quandl call
    code = 'FNYX_' + ticker
    q_code = 'FINRA/' + code
    
    # Create a row of all the data for that row so I can access it easily
    filtered_row = finra_df[finra_df['code']==code]
    
    # Use Quandl to grab the short volumne info based on the code and relevant dates
    data = quandl.get(q_code, start_date=filtered_row['from_date'], end_date=filtered_row['to_date'])
    
    # Add a ticker column to data
    data['Ticker']=ticker
    
    # Append the data grabbed to short_df
    short_df = short_df.append(data)

In [59]:
short_df['Ticker'].value_counts(ascending=False)

KeyError: 'Ticker'

In [55]:
display(short_df.head())

,ShortVolume,ShortExemptVolume,TotalVolume,Ticker
Date,,,,
2013-04-01,11424.0,0.0,17954.0,MMM
2013-04-02,29303.0,0.0,48204.0,MMM
2013-04-03,22121.0,0.0,35692.0,MMM
2013-04-04,21142.0,0.0,32433.0,MMM
2013-04-05,13000.0,0.0,23385.0,MMM


In [57]:
short_df.reset_index(inplace=True)
short_df = short_df.set_index(['Ticker','Date'])

short_df

ShortVolume  ShortExemptVolume  TotalVolume
Ticker Date                                                   
MMM    2013-04-01      11424.0                0.0      17954.0
       2013-04-02      29303.0                0.0      48204.0
       2013-04-03      22121.0                0.0      35692.0
       2013-04-04      21142.0                0.0      32433.0
       2013-04-05      13000.0                0.0      23385.0
...                        ...                ...          ...
ZTS    2020-04-20      61849.0                0.0     113772.0
       2020-04-21      47023.0              953.0      88102.0
       2020-04-22      36361.0               33.0     117974.0
       2020-04-23      77480.0              200.0     174712.0
       2020-04-24      21032.0              100.0      86190.0

[846566 rows x 3 columns]

In [60]:
# Calculate the short % of volume
short_df['ShortPct']=short_df['ShortVolume'] / short_df['TotalVolume']

# Drop columns we don't need
final_df = short_df.drop(['ShortVolume', 'ShortExemptVolume', 'TotalVolume'], axis=1)

ShortPct
Ticker Date                
MMM    2013-04-01  0.636293
       2013-04-02  0.607896
       2013-04-03  0.619775
       2013-04-04  0.651867
       2013-04-05  0.555912

In [61]:
# Check on final_df so far
display(final_df)

ShortPct
Ticker Date                
MMM    2013-04-01  0.636293
       2013-04-02  0.607896
       2013-04-03  0.619775
       2013-04-04  0.651867
       2013-04-05  0.555912
...                     ...
ZTS    2020-04-20  0.543622
       2020-04-21  0.533734
       2020-04-22  0.308212
       2020-04-23  0.443473
       2020-04-24  0.244019

[846566 rows x 1 columns]

In [64]:
# Now to gather the historical stock prices for these companies
import yfinance as yf

In [ ]:
# Set start and end dates for my loops
start = '2013-04-01'
end = '2020-08-28'

# Initialize a price dataframe
price_df = pd.DataFrame()

# Look through my sample_df 
for ticker in tickers:
    try:
        data = yf.download(ticker,
                          start=start,
                          end=end,
                          progress=False)
        data['Ticker']=ticker
        price_df = price_df.append(data)
    except:
        continue


1 Failed download:
- BRK_B: No data found, symbol may be delisted
